In [ ]:
%matplotlib inline
TBP_DCM_PATH="/scratch/tb/cxr/"
INDIANA_DCM_PATH="/scratch/tb/NLM_Indiana_CXRs/cxr/"

NUMBER_SAMPLES=1000
RANDOM_SEED=13

SAMPLE_IMAGE_SIZE=64

In [ ]:
from glob import glob
import random
import functools
import multiprocessing

from functools import partial
import SimpleITK as sitk

import pandas

import numpy as np
from numpy import linalg as LA


import matplotlib.pyplot as plt

from myshow import myshow

import pickle

In [ ]:
from importlib import reload  
from tbpcxr.model import PCAModel
#reload(tbpcxr.model)
#PCAModel = tbpcxr.model.PCAModel

from tbpcxr.utilities import read_dcm, normalize_img
import tbpcxr.registration 

In [ ]:
def read_and_classify(cxr_model, filename):
    try:
        img = read_dcm(filename)
    except:
        img = sitk.ReadImage(filename, sitk.sitkFloat32)

    nimg = normalize_img(img)

    rimg = cxr_model.register_to_atlas_and_resample(nimg, verbose=0)
    arr = cxr_model._images_to_arr([rimg])

    return cxr_model.outlier_predictor(arr)[0], rimg

def tile_with_classification(image_list, class_list, width=10): 
    
    def img_to_classification(img, c):
        img = sitk.RescaleIntensity(img, outputMinimum=0, outputMaximum=255) 
        img = sitk.Cast(img, sitk.sitkUInt8)
    
        img2 = img//2
    
        if (c < 0 ):
            img = sitk.Compose(img, img2, img2)
        else:
            img = sitk.Compose(img2, img, img2)
        return img
    
    return sitk.Tile([ img_to_classification(img, c) for img,c  in zip(image_list, class_list)], [width,0])



In [ ]:
pkl_filename = "cxr_model.pkl"
with open(pkl_filename, 'rb') as file:
    cxr_model = pickle.load(file)

In [ ]:
indiana_file_list =  glob(INDIANA_DCM_PATH+"/**/*.dcm", recursive=True)
print( "Found {0} DICOM.".format(len(indiana_file_list)))

read_and_classify_p = partial(read_and_classify, cxr_model)

with multiprocessing.Pool() as pool:
    indiana_class_rimg = pool.map(read_and_classify_p, indiana_file_list[:10])


In [ ]:
outlier_classification, rimg_set = list(zip(*indiana_class_rimg))

print(indiana_class_rimg[0][0][0])
#myshow( tile_with_classification(outlier_classification, rimg_set, 50))

In [ ]:
test_file_list = dcm_list # random.sample(dcm_list, NUMBER_SAMPLES)



with multiprocessing.Pool() as pool:
    test_set = pool.map(read_and_normalize, test_file_list[:10])

In [ ]:
dcm_list = glob(TBP_DCM_PATH+"/**.dcm")

print( "Found {0} DICOM.".format(len(dcm_list)))

In [ ]:
reg_set = []
outlier_list = []
for nimg in test_set:
    
    #img = read_dcm(fname)
    #nimg = normalize_img(img)
    rimg = cxr_model.register_to_atlas_and_resample(nimg, verbose=0)
    arr = cxr_model._images_to_arr([rimg])
    
    
    outlier_list.append(cxr_model.outlier_predictor(arr))
    reg_set.append(rimg)
    
   

In [ ]:
myshow( tile_with_classification(test_set, outlier_list, 50))

In [ ]:
for img in test_set:
    rimg = pca.register_to_atlas_and_resample(img)
    pca._images_to_arr([img])
    
    outlier_predictor
test_reg_set = [register_and_resample(fixed=normalize_img(pca.avg()), moving=img) for img in test_set]

avg = sitk.NaryAdd(train_reg_set )/len(train_reg_set )

In [ ]:
myshow(avg)
myshow(normalize_img(pca.avg()))
sitk.Show(tile_with_scale(train_reg_set))

In [ ]:
test_reg_set = train_reg_set
test_vec = pca._images_to_arr(test_reg_set)

In [ ]:
res = pca.residuals(test_vec)

In [ ]:
mad = pca.median_absolute_deviations(test_vec)

In [ ]:
idxs = np.argsort(mad)
sitk.Show(tile_with_scale([test_set[i] for i in idxs if mad[i] > 6], 10) , title="Test Registered Images Sorted MAD")



In [ ]:
idxs = np.argsort(res)
sitk.Show(tile_with_scale([test_set[i] for i in idxs], 10) , title="Test Registered Images Sorted residuals")



In [ ]:
%history -g